# Automatic Documentation with GitHub Workflow

It's possible to automate the generation of SDK files using [GitHub Workflows](https://docs.github.com/en/actions/using-workflows).

## Workflow

This workflow is titled "Automated Documentation and Code Formatting" and is specifically designed to automate the generation of documentation based on Notebooks. It is triggered by a push to the repository or manually through workflow_dispatch.

In [ ]:
# .github/workflows

name: Automated Documentation and Code Formatting

# Workflow triggers
on:
  workflow_dispatch:  # Allows manual triggering of the workflow
  push:               # Triggers the workflow on every push to the repository

jobs:
  format-code-customized:
    runs-on: ubuntu-latest  # Specifies the virtual machine to use, in this case, the latest version of Ubuntu

    # Defines permissions for this job
    permissions:
      contents: write  # Permissions to write to the repository

    steps:
      - uses: actions/checkout@v4  # Checks out the code from the repository

      # Step to update and prepare the documentation
      - name: Prepare and Update Documentation

        run: |

          # Pulls necessary Docker images for documentation
          docker pull dunderlab/docs
          docker pull sphinxdoc/sphinx-latexpdf

          # Installs required Python packages
          pip install nbsphinx
          pip install dunderlab-docs

          # Sets up initial documentation if a 'docs' directory does not exist
          if [ ! -d "docs" ]; then
              dunderlab_docs quickstart '--project "${{ vars.PROJECT_NAME }}" --author "${{ vars.AUTHOR }}" --extensions nbsphinx,dunderlab.docs --no-batchfile --quiet --sep'
          fi

          # Generates API documentation and builds HTML and Latex PDF if SUBMODULE is set
          if [ -n "${{ vars.SUBMODULE }}" ]; then
            dunderlab_docs apidoc "${{ vars.MODULE }}"
            dunderlab_docs build html "${{ vars.MODULE }}/${{ vars.SUBMODULE }}"
            dunderlab_docs build latexpdf "${{ vars.MODULE }}/${{ vars.SUBMODULE }}"
          else
            dunderlab_docs apidoc "${{ vars.MODULE }}"
            dunderlab_docs build html "${{ vars.MODULE }}"
            dunderlab_docs build latexpdf "${{ vars.MODULE }}"
          fi

          # Adds a configuration file for Read the Docs
          # Verifies if .readthedocs.yml exists, if not, creates it
          if [ ! -f ".readthedocs.yml" ]; then
            echo """# .readthedocs.yml
            # Read the Docs configuration file
            # See https://docs.readthedocs.io/en/stable/config-file/v2.html for details

            # Required
            version: 2

            # Build documentation in the docs/ directory with Sphinx
            sphinx:
              configuration: docs/source/conf.py

            # Optionally set the version of Python and requirements required to build your docs
            python:
              install:
                - requirements: docs/requirements

            # Set the version of Python and other tools you might need
            build:
              os: ubuntu-22.04
              tools:
                python: \"3.11\"

            formats:
              - epub
              - pdf
            """ >> .readthedocs.yml
          fi


      # Commit all changed files back to the repository
      - uses: stefanzweifel/git-auto-commit-action@v5

## Integration with Read the Docs
The workflow adds a `.readthedocs.yml` file to configure automatic documentation building on the Read the Docs platform, specifying details such as the Python version and required tools.

## Repository Variables

The following variables should be defined in the GitHub repository settings to ensure the proper functioning of the workflow. These variables are used within the workflow to customize the documentation and code formatting process. For more information on setting up and using variables in GitHub Actions, refer to [GitHub Variables Documentation](https://docs.github.com/actions/learn-github-actions/variables).

![Read the Docs](_images/Screenshot_20240205_193016.png "GitHub Actions secrets and variables")


### Mandatory Variables
1. `PROJECT_NAME`: The name of the project. This is used to identify the project within the documentation.
2. `AUTHOR`: The name of the author or maintainer of the project. This is used for attribution in the documentation.
3. `MODULE`: The specific module within the repository for which the documentation is to be generated. This is crucial for the correct generation of API documentation and other related materials.

### Optional Variables
These variables are not mandatory but can be used to provide additional control and customization of the workflow.

1. `SUBMODULE`: Specifies a particular submodule within the main module for which the documentation should be specifically generated. This allows for more granular control over the documentation process.
2. `NO_COMPILE`: A boolean flag (`True`/`False`). When set to `True`, it prevents the compilation of certain documentation formats (like HTML or Latex PDF). This can be useful for speeding up the workflow when full documentation generation is not needed.
